## 1. Parse all links from Wikipedia xml dump

In [1]:
import xml.etree.ElementTree as etree
import codecs
import csv
import time
import os
import re
import gzip
import pandas as pd
import numpy as np

In [2]:
PATH_WIKI_XML = '/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/ukwiki_20180920/'
PATH_WIKI_OUT = '/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/ukwiki_20180920/'

In [3]:
PATH_TO_DATA = '/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/ukwiki_20180920/'
data_time = '20180920'

In [4]:
def unpack(file_name):
    file_name_new = file_name.replace(".gz","")
    with gzip.open(file_name, 'rb') as f_in, open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    return file_name_new

In [5]:
def pack_and_remove(file_name):
    file_name_new = file_name+'.gz'
    with open(file_name, 'rb') as f_in, gzip.open(file_name_new, 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(file_name)
    return file_name_new

In [6]:
WIKI_FILENAMES = []
for file in os.listdir(PATH_WIKI_XML):
    if file.endswith("pages-meta-current.xml"):
        WIKI_FILENAMES.append(file)
print(WIKI_FILENAMES)

['ukwiki-20180920-pages-meta-current.xml']


In [ ]:
ENCODING = "utf-8"


# Nicely formatted time string
def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


def strip_tag_name(t):
    t = elem.tag
    idx = k = t.rfind("}")
    if idx != -1:
        t = t[idx + 1:]
    return t


totalCount = 0
articleCount = 0
redirectCount = 0
total_article_text_len = 0

In [ ]:
start_time = time.time()
regex_links = re.compile(r"\[\[(?P<article>(?!.*?\:).*?)(?:\{\{.*\}\})?(?:\|(?P<text>(?!.*?\:).*?))?\]\]")
for WikiXML in WIKI_FILENAMES:
    pathWikiXML = os.path.join(PATH_WIKI_XML, WikiXML)
    pathArticles = os.path.join(PATH_WIKI_OUT, WikiXML+"_art.csv")
    pathArticlesRedirect = os.path.join(PATH_WIKI_OUT, WikiXML+"_red.csv")
    with codecs.open(pathArticles, "w", ENCODING) as articlesFH, \
    codecs.open(pathArticlesRedirect, "w", ENCODING) as redirectFH:
        articlesWriter = csv.writer(articlesFH, quoting=csv.QUOTE_MINIMAL)
        articlesWriter.writerow(['id', 'title', 'text_len', 'link_pos', 'link_val', 'link_txt']) 
        redirectWriter = csv.writer(redirectFH, quoting=csv.QUOTE_MINIMAL)
        redirectWriter.writerow(['id', 'title', 'redirect'])
        for event, elem in etree.iterparse(pathWikiXML, events=('start', 'end')):
            tname = strip_tag_name(elem.tag)

            if event == 'start':
                if tname == 'page':
                    title = ''
                    id = -1
                    redirect = ''
                    inrevision = False
                    ns = 0
                    article_text_len = 0
                    links = []
                elif tname == 'revision':
                    # Do not pick up on revision id's
                    inrevision = True
            else:
                if tname == 'title':
                    title = elem.text
                elif tname == 'id' and not inrevision:
                    id = int(elem.text)
                elif tname == 'redirect':
                    redirect = elem.attrib['title']
                elif tname == 'ns':
                    ns = int(elem.text)
                elif tname == 'page' and ns == 0:
                    totalCount += 1
                        
                    if len(redirect) == 0:
                        articleCount += 1
                        total_article_text_len += article_text_len
                        if len(links) == 0:
                            articlesWriter.writerow([id, title, article_text_len, 0, "", ""])
                        for link in links:
                            articlesWriter.writerow([id, title, article_text_len, link[0], link[1], link[2]])
                    else:
                        redirectCount += 1
                        redirectWriter.writerow([id, title, redirect])

                    if totalCount > 1 and (totalCount % 100000) == 0:
                        print("{:,}".format(totalCount))
                elif tname == 'text' and elem.text != None:
                    article_text_len = len(elem.text)
                    for match in regex_links.finditer(elem.text):    
                        link_pos = match.start()
                        link_title = match.group("article")
                        link_title = link_title.replace("&nbsp;", " ")
                        link_title = link_title.replace("&ndash;", "-")
                        link_title = link_title.replace("&mdash;", "—")
                        link_title = link_title.replace("%20", " ")
                        
                        link_text = match.group("text")
                        links.append((link_pos,link_title,link_text))


                elem.clear()
    
    fn_in = pathArticles
    with open(fn_in, 'rb') as f_in, gzip.open(fn_in+'.gz', 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(fn_in)
    
    fn_in = pathArticlesRedirect
    with open(fn_in, 'rb') as f_in, gzip.open(fn_in+'.gz', 'wb') as f_out:
        f_out.writelines(f_in)
    os.remove(fn_in)
    
    elapsed_time = time.time() - start_time
    print("File processed: {}".format(WikiXML))
    print("Total pages: {:,}".format(totalCount))
    print("Article pages: {:,}".format(articleCount))
    print("Redirect pages: {:,}".format(redirectCount))
    print("Total article lenght: {:,}".format(total_article_text_len))
    print("Elapsed time: {}".format(hms_string(elapsed_time)))

elapsed_time = time.time() - start_time

print("Total pages: {:,}".format(totalCount))
print("Article pages: {:,}".format(articleCount))
print("Redirect pages: {:,}".format(redirectCount))
print("Total article lenght: {:,}".format(total_article_text_len))
print("Elapsed time: {}".format(hms_string(elapsed_time)))

In [ ]:
### results in two csv.gz files - with articles and redirections

In [7]:
df_articles = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-pages-meta-current.xml_red.csv.gz", encoding='UTF-8', quotechar="\"")

In [8]:
df_articles.head()

,id,title,redirect
0,2,Esperanto,Есперанто
1,6,Wikipedia,Вікіпедія
2,9,HomePage,Головна сторінка
3,647,Капустянко Микола,Капустянський Микола Олександрович
4,693,Володимир Винниченко,Винниченко Володимир Кирилович


In [63]:
df_articles[df_articles['title']=='Католицтво']

,id,title,redirect
451080,2808027,Католицтво,Католицька церква


In [ ]:
# wrong parsing:
# 'Свідницький повіт (Нижньосілезьке воєводство){{!}}Свідницький повіт'
# japanese symbols 

In [ ]:
# find where a bug - !redirections: 'католицизм', 'королева', 'ВВС Україна'...

# found:
# 'королева', 'Королева' not in redirection list - created after dump, was a red link then
# 'католицизм', 'ВВС Україна' - redirections to redirections

## 1.1 Remove redirections to redirections bug

In [9]:
df_articles = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-pages-meta-current.xml_red.csv.gz",
                          encoding='UTF-8', quotechar="\"")
df_articles.head()

,id,title,redirect
0,2,Esperanto,Есперанто
1,6,Wikipedia,Вікіпедія
2,9,HomePage,Головна сторінка
3,647,Капустянко Микола,Капустянський Микола Олександрович
4,693,Володимир Винниченко,Винниченко Володимир Кирилович


In [7]:
df_redirect1 = df_articles[['title']]
df_redirect1.columns = ['redirect']

In [8]:
df_redirect1.head()

,redirect
0,Esperanto
1,Wikipedia
2,HomePage
3,Капустянко Микола
4,Володимир Винниченко


In [9]:
df_redirect2 = df_articles[['redirect']]

In [10]:
df_redirect2.head()

,redirect
0,Есперанто
1,Вікіпедія
2,Головна сторінка
3,Капустянський Микола Олександрович
4,Винниченко Володимир Кирилович


In [11]:
frames = [df_redirect1, df_redirect2]
df_all_redirections = pd.concat(frames)

In [12]:
df_all_redirections

,redirect
0,Esperanto
1,Wikipedia
2,HomePage
3,Капустянко Микола
4,Володимир Винниченко
5,Белза Ігор
6,Інформаційна технологія
7,Історик
8,Міфологія давньої Греції
9,Історія філософії в особах


In [13]:
df_all_redirections = df_all_redirections.drop_duplicates()

In [14]:
df_all_redirections.shape

(723894, 1)

In [16]:
df_all_redirections.to_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-redirections.csv", 
                     index = False, encoding='UTF-8', quotechar="\"", sep ="^")

## 2. Get all article names

In [8]:
UKWIKI_ART_FNMS = []
for file in os.listdir(PATH_TO_DATA):
    if re.match(r"ukwiki-" + data_time + "-pages-meta-current.xml_art.csv.gz", file):
        UKWIKI_ART_FNMS.append(file)
print('UKWIKI_ART_FNMS:', UKWIKI_ART_FNMS)

UKWIKI_ART_FNMS: ['ukwiki-20180920-pages-meta-current.xml_art.csv.gz']


In [9]:
df_uk_id_name = None
for fn in UKWIKI_ART_FNMS:
    fn = PATH_TO_DATA+fn
    print(fn)
    fn_new = unpack(fn)
    df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
    print(df_articles.head(100))
    df_id_name_tmp = df_articles[['id', 'title', 'text_len']].drop_duplicates()
    df_id_name_tmp.columns = ['id', 'title', 'length']
    #print("df_id_name_tmp size: {}".format(df_id_name_tmp.shape))
    if df_uk_id_name is not None:
        df_uk_id_name = df_uk_id_name.append(df_id_name_tmp)        
        #print("append")
    else:
        df_uk_id_name = df_id_name_tmp
        #print("assign")
    print("df_uk_id_name size: {}".format(df_uk_id_name.shape))
    os.remove(fn_new)
df_uk_id_name = df_uk_id_name.drop_duplicates()

/media/andrii/earth/Katia/BabelNet_baseline/uk-en_redlinks/data/ukwiki-20180920-pages-meta-current.xml_art.csv.gz
     id             title  text_len  link_pos                        link_val  \
0     3  Головна сторінка      3197         0                             NaN   
1    13         Географія     16843      1228              Об'єкт (філософія)   
2    13         Географія     16843      1533                       Ератосфен   
3    13         Географія     16843      1599                         географ   
4    13         Географія     16843      1611                Клавдій Птолемей   
5    13         Географія     16843      1722                     Відродження   
6    13         Географія     16843      1797                        еллінізм   
7    13         Географія     16843      1883                 Герард Меркатор   
8    13         Географія     16843      1996        Александер фон Гумбольдт   
9    13         Географія     16843      2047                     Карл Рітте

In [35]:
df_uk_id_name.to_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", 
                     index = False, encoding='UTF-8', quotechar="\"", sep ="^")

## 4. Get links between all pages

In [10]:
df_uk_redirections = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-redirections.csv", 
                            encoding='UTF-8', quotechar="\"", sep ="^")
df_uk_redirections.columns = ['title']
df_uk_redirections.head()

,title
0,Esperanto
1,Wikipedia
2,HomePage
3,Капустянко Микола
4,Володимир Винниченко


In [11]:
print(df_uk_redirections.shape)

(723893, 1)


In [12]:
df_uk_id_name = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", 
                         encoding='UTF-8', quotechar="\"", sep ="^")

In [13]:
df_uk_name = df_uk_id_name[['title']]
df_uk_name.head()

,title
0,Головна сторінка
1,Географія
2,Атом
3,Мільярд
4,Ядро


In [14]:
frames = [df_uk_redirections, df_uk_name]
df_uk_name_all = pd.concat(frames)

In [15]:
df_uk_name_all['title'] = df_uk_name_all['title'].str.strip()

In [16]:
df_uk_name_all.head()

,title
0,Esperanto
1,Wikipedia
2,HomePage
3,Капустянко Микола
4,Володимир Винниченко


In [17]:
df_uk_name_all_strip = df_uk_name_all.iloc[:,0]

In [18]:
df_uk_name_all_strip = df_uk_name_all_strip.str.replace('  ', ' ')

In [19]:
df_uk_name_all_stripped = df_uk_name_all_strip.to_frame()

In [20]:
df_uk_name_all_stripped.head()

,title
0,Esperanto
1,Wikipedia
2,HomePage
3,Капустянко Микола
4,Володимир Винниченко


In [21]:
df_uk_name_all_stripped = df_uk_name_all_stripped.drop_duplicates()

In [22]:
df_uk_name_all.shape

(1543105, 1)

In [23]:
df_uk_name_all['title_lc'] = df_uk_name_all['title'].str.lower()

In [24]:
df_uk_name_all.head()

,title,title_lc
0,Esperanto,esperanto
1,Wikipedia,wikipedia
2,HomePage,homepage
3,Капустянко Микола,капустянко микола
4,Володимир Винниченко,володимир винниченко


In [25]:
fn = PATH_TO_DATA+"ukwiki-" + data_time + "-pages-meta-current.xml_art.csv.gz"
print(fn)
fn_new = unpack(fn)
df_articles = pd.read_csv(fn_new, encoding='UTF-8', quotechar="\"")
df_articles['link_val'] = df_articles['link_val'].str.split("#").str[0] # ???
df_articles = df_articles[df_articles['link_val'].str.strip() != ''] 
df_articles.head(100)

/media/andrii/earth/Katia/CS_MasterThesis/BabelNet_baseline/uk-en_redlinks/data/ukwiki_20180920/ukwiki-20180920-pages-meta-current.xml_art.csv.gz


,id,title,text_len,link_pos,link_val,link_txt
0,3,Головна сторінка,3197,0,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт
2,13,Географія,16843,1533,Ератосфен,NaN
3,13,Географія,16843,1599,географ,NaN
4,13,Географія,16843,1611,Клавдій Птолемей,NaN
5,13,Географія,16843,1722,Відродження,NaN
6,13,Географія,16843,1797,еллінізм,NaN
7,13,Географія,16843,1883,Герард Меркатор,Герарда Меркатора
8,13,Географія,16843,1996,Александер фон Гумбольдт,Олександр Гумбольдт
9,13,Географія,16843,2047,Карл Ріттер,NaN


In [26]:
df_articles['title'] = df_articles['title'].str.strip()
df_articles['link_val'] = df_articles['link_val'].str.strip()

In [27]:
df_uk_titles_series = df_articles.iloc[:,4]

In [28]:
df_uk_titles_series_stripped = df_uk_titles_series.str.replace('  ', ' ')

In [29]:
df_titles_stripped = df_uk_titles_series_stripped.to_frame()

In [30]:
df_titles_stripped.head()

,link_val
0,NaN
1,Об'єкт (філософія)
2,Ератосфен
3,географ
4,Клавдій Птолемей


In [31]:
df_titles_stripped['link_val'] = df_titles_stripped['link_val'].str.lower()

In [33]:
print(len(df_titles_stripped))
print(len(df_articles))

22650865
22650865


# Create files with page links

In [35]:
df_articles_with_lc = pd.concat([df_articles, df_titles_stripped], axis=1)

In [36]:
df_articles_with_lc.head()

,id,title,text_len,link_pos,link_val,link_txt,link_val
0,3,Головна сторінка,3197,0,NaN,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт,об'єкт (філософія)
2,13,Географія,16843,1533,Ератосфен,NaN,ератосфен
3,13,Географія,16843,1599,географ,NaN,географ
4,13,Географія,16843,1611,Клавдій Птолемей,NaN,клавдій птолемей


In [95]:
df_uk_id_name = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", 
                         encoding='UTF-8', quotechar="\"", sep ="^")

In [96]:
df_uk_id_name = df_uk_id_name[['id', 'title']]

In [97]:
df_uk_id_name.head()

,id,title
0,3,Головна сторінка
1,13,Географія
2,584,Атом
3,585,Мільярд
4,586,Ядро


In [91]:
redirections = PATH_TO_DATA+'ukwiki-' + data_time + '-pages-meta-current.xml_red.csv.gz'
redirections = unpack(redirections)

In [92]:
df_uk_redirections = pd.read_csv(redirections, encoding='UTF-8', quotechar="\"")

In [93]:
df_uk_redirections = df_uk_redirections[['id', 'title']]

In [94]:
df_uk_redirections.head()

,id,title
0,2,Esperanto
1,6,Wikipedia
2,9,HomePage
3,647,Капустянко Микола
4,693,Володимир Винниченко


In [98]:
df_all_ids_uk = pd.concat([df_uk_id_name, df_uk_redirections], axis=0)

In [99]:
df_all_ids_uk.head()

,id,title
0,3,Головна сторінка
1,13,Географія
2,584,Атом
3,585,Мільярд
4,586,Ядро


In [100]:
df_all_ids_uk.shape

(1272760, 2)

In [101]:
df_all_ids_uk['title'] = df_all_ids_uk['title'].str.strip()

df_all_ids_uk_series = df_all_ids_uk.iloc[:,1]

df_all_ids_uk_series_stripped = df_all_ids_uk_series.str.replace('  ', ' ')
df_all_ids_uk_stripped = df_all_ids_uk_series_stripped.to_frame()

df_all_ids_uk_stripped['title'] = df_all_ids_uk_stripped['title'].str.lower()

In [107]:
df_all_ids_uk_stripped.shape

(1272760, 1)

In [108]:
df_all_ids_uk_cleaned = pd.concat([df_all_ids_uk, df_all_ids_uk_stripped], axis=1)

In [113]:
df_all_ids_uk_cleaned.columns = ['id', 'title', 'title_lc']

In [114]:
df_all_ids_uk_cleaned.head()

,id,title,title_lc
0,3,Головна сторінка,головна сторінка
1,13,Географія,географія
2,584,Атом,атом
3,585,Мільярд,мільярд
4,586,Ядро,ядро


In [116]:
df_articles_with_lc.columns = ['id', 'title', 'text_len', 'link_pos', 'link_val', 'link_txt', 'link_val_lc']

In [117]:
df_articles_with_lc.head()

,id,title,text_len,link_pos,link_val,link_txt,link_val_lc
0,3,Головна сторінка,3197,0,NaN,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт,об'єкт (філософія)
2,13,Географія,16843,1533,Ератосфен,NaN,ератосфен
3,13,Географія,16843,1599,географ,NaN,географ
4,13,Географія,16843,1611,Клавдій Птолемей,NaN,клавдій птолемей


In [119]:
page_links_dirty = pd.merge(df_articles_with_lc, df_all_ids_uk_cleaned, how='left', left_on=['link_val_lc'], right_on=['title_lc'])

In [121]:
page_links_dirty = page_links_dirty[pd.notnull(page_links_dirty['link_val'])]

In [135]:
page_links_dirty.head()

,id_x,title_x,text_len,link_pos,link_val,link_txt,link_val_lc,id_y,title_y,title_lc
3,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт,об'єкт (філософія),884280.0,Об'єкт (філософія),об'єкт (філософія)
4,13,Географія,16843,1533,Ератосфен,NaN,ератосфен,21623.0,Ератосфен,ератосфен
5,13,Географія,16843,1599,географ,NaN,географ,274944.0,Географ,географ
6,13,Географія,16843,1611,Клавдій Птолемей,NaN,клавдій птолемей,26194.0,Клавдій Птолемей,клавдій птолемей
7,13,Географія,16843,1722,Відродження,NaN,відродження,20459.0,Відродження,відродження


In [136]:
page_links_with_titles = page_links_dirty[['id_x', 'title_x', 'id_y', 'link_val', 'title_lc']]

In [137]:
page_links_with_titles.head()

,id_x,title_x,id_y,link_val,title_lc
3,13,Географія,884280.0,Об'єкт (філософія),об'єкт (філософія)
4,13,Географія,21623.0,Ератосфен,ератосфен
5,13,Географія,274944.0,географ,географ
6,13,Географія,26194.0,Клавдій Птолемей,клавдій птолемей
7,13,Географія,20459.0,Відродження,відродження


In [138]:
page_links_with_titles.to_csv(PATH_TO_DATA+'page_links_with_titles.csv', index = False, encoding='UTF-8', sep='^')

In [126]:
page_links_ids = page_links_with_titles[['id_x', 'id_y']]
page_links_ids.columns = ['id', 'link_id']

In [128]:
page_links_ids.to_csv(PATH_TO_DATA+'uk_page_links_ids.csv', index = False, encoding='UTF-8', sep='^')

In [129]:
# point red and blue links in page links

In [165]:
page_links_with_titles.head()

,id_x,title_x,id_y,link_val,title_lc
3,13,Географія,884280.0,Об'єкт (філософія),об'єкт (філософія)
4,13,Географія,21623.0,Ератосфен,ератосфен
5,13,Географія,274944.0,географ,географ
6,13,Географія,26194.0,Клавдій Птолемей,клавдій птолемей
7,13,Географія,20459.0,Відродження,відродження


In [154]:
all_uk_redlinks_with_parent_ids = pd.read_csv(PATH_TO_DATA+'all_uk_redlinks_with_parent_ids.csv', encoding='UTF-8', quotechar="\"")

In [155]:
all_uk_redlinks_with_parent_ids = all_uk_redlinks_with_parent_ids[['red_link_name']]

In [158]:
all_uk_redlinks_with_parent_ids['red_link_name'] = all_uk_redlinks_with_parent_ids['red_link_name'].str.strip()

all_uk_redlinks_series = all_uk_redlinks_with_parent_ids.iloc[:,0]

all_uk_redlinks_series_stripped = all_uk_redlinks_series.str.replace('  ', ' ')
all_uk_redlinks_stripped = all_uk_redlinks_series_stripped.to_frame()

all_uk_redlinks_stripped['red_link_name'] = all_uk_redlinks_stripped['red_link_name'].str.lower()

In [162]:
all_uk_redlinks = all_uk_redlinks_stripped.drop_duplicates()

In [164]:
all_uk_redlinks.head()

,red_link_name
0,географія розвитку
1,географія часу
3,туристична географія
4,рентгенівська спектроскопія поглинання
5,рентгенівська фото електронна спектроскопія


In [167]:
page_links_with_red = pd.merge(page_links_with_titles, all_uk_redlinks, how='left', left_on=['title_lc'], right_on=['red_link_name'])

In [ ]:
page_links_with_red

In [48]:
# find red links among all links
links_which_exist = pd.merge(df_uk_name_all, df_titles_stripped,
                             how='right', left_on=['title_lc'], right_on=['link_val'])

In [49]:
links_which_exist = links_which_exist.drop_duplicates()

In [50]:
links_which_exist = links_which_exist.dropna(how='all')

In [51]:
links_which_exist

,title,title_lc,link_val
0,Esperanto,esperanto,esperanto
2,Wikipedia,wikipedia,wikipedia
31,Володимир Винниченко,володимир винниченко,володимир винниченко
155,Белза Ігор,белза ігор,белза ігор
157,Інформаційна технологія,інформаційна технологія,інформаційна технологія
162,Історик,історик,історик
2772,Міфологія давньої Греції,міфологія давньої греції,міфологія давньої греції
2776,Леонід Глібов,леонід глібов,леонід глібов
2806,США,сша,сша
56900,WWW,www,www


In [52]:
candidates_for_redlinks = links_which_exist[links_which_exist['title'].isnull()]

In [53]:
candidates_for_redlinks = candidates_for_redlinks[['link_val']]

In [54]:
candidates_for_redlinks.shape

(1532585, 1)

In [55]:
candidates_for_redlinks.head()

,link_val
31758551,географія розвитку
31758552,географія часу
31758555,туристична географія
31758556,рентгенівська спектроскопія поглинання
31758557,рентгенівська фото електронна спектроскопія


In [56]:
candidates_for_redlinks[candidates_for_redlinks['link_val']=='москва ']

,link_val


In [57]:
df_articles.head()

,id,title,text_len,link_pos,link_val,link_txt
0,3,Головна сторінка,3197,0,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт
2,13,Географія,16843,1533,Ератосфен,NaN
3,13,Географія,16843,1599,географ,NaN
4,13,Географія,16843,1611,Клавдій Птолемей,NaN


In [58]:
df_uk_titles_series = df_articles.iloc[:,4]

In [59]:
df_uk_titles_series_stripped = df_uk_titles_series.str.replace('  ', ' ')

In [60]:
df_titles_stripped = df_uk_titles_series_stripped.to_frame()

In [61]:
df_titles_stripped['link_val_lc'] = df_titles_stripped['link_val'].str.lower()

In [62]:
df_pagelinks_with_lowercase = pd.concat([df_articles, df_titles_stripped], axis=1)

In [63]:
df_pagelinks_with_lowercase_redlinks = pd.merge(df_pagelinks_with_lowercase, candidates_for_redlinks,
         how='left', left_on=['link_val_lc'], right_on=['link_val'])

In [64]:
df_pagelinks_with_lowercase_redlinks

,id,title,text_len,link_pos,link_val_x,link_txt,link_val_x,link_val_lc,link_val_y
0,3,Головна сторінка,3197,0,NaN,NaN,NaN,NaN,NaN
1,13,Географія,16843,1228,Об'єкт (філософія),Об'єкт,Об'єкт (філософія),об'єкт (філософія),NaN
2,13,Географія,16843,1533,Ератосфен,NaN,Ератосфен,ератосфен,NaN
3,13,Географія,16843,1599,географ,NaN,географ,географ,NaN
4,13,Географія,16843,1611,Клавдій Птолемей,NaN,Клавдій Птолемей,клавдій птолемей,NaN
5,13,Географія,16843,1722,Відродження,NaN,Відродження,відродження,NaN
6,13,Географія,16843,1797,еллінізм,NaN,еллінізм,еллінізм,NaN
7,13,Географія,16843,1883,Герард Меркатор,Герарда Меркатора,Герард Меркатор,герард меркатор,NaN
8,13,Географія,16843,1996,Александер фон Гумбольдт,Олександр Гумбольдт,Александер фон Гумбольдт,александер фон гумбольдт,NaN
9,13,Географія,16843,2047,Карл Ріттер,NaN,Карл Ріттер,карл ріттер,NaN


In [65]:
redlinks_with_all = df_pagelinks_with_lowercase_redlinks[~(df_pagelinks_with_lowercase_redlinks['link_val_y'].isnull())]

In [66]:
redlinks = redlinks_with_all[['id', 'link_val_x']]
redlinks.columns = ['id', 'link_val', 'dd']

In [67]:
redlinks = redlinks[['id', 'link_val']]

In [68]:
redlinks.to_csv(PATH_TO_DATA+'ukwiki-20180920-all_reslinks.csv', index = False, encoding='UTF-8', quotechar="\"")

In [40]:
df_articles.to_csv(PATH_TO_DATA+'ukwiki-20180920-pages-links.csv.gz', index = False, encoding='UTF-8', quotechar="\"")

In [41]:
pack_and_remove(fn_new)

'/media/andrii/earth/Katia/BabelNet_baseline/uk-en_redlinks/data/ukwiki-20180920-pages-meta-current.xml_art.csv.gz'

## 5. Get En correspondences for Ukrainian articles

In [ ]:
# 1) convert sql dump into csv file  
# see https://stackoverflow.com/questions/27584405/how-to-import-a-mysqldump-into-pandas

from io import StringIO
import re, shutil
import os.path

reg1 = re.compile('`([\w_]+)`')
reg2 = re.compile(r'\((?P<val>\d+?,\'[a-z]*?\',\'.*?(?:(?:\\\\)|(?<!\\))\')\)')

FN_UK_LANGLINKS = 'ukwiki-20180920-langlinks.sql'
dump_filename = PATH_TO_DATA+FN_UK_LANGLINKS
target_table = "langlinks"

sio = StringIO()

read_mode = 0 # 0 - skip, 1 - header, 2 - data
with open(dump_filename, 'r', encoding='ISO-8859-1') as f:
    for line in f:
        line = line.strip()
        if line.lower().startswith('insert') and target_table in line:
            read_mode = 2
        if line.lower().startswith('create table') and target_table in line:
            read_mode = 1
            continue

        if read_mode==0:
            continue

        # Filling up the headers
        elif read_mode==1:
            if line.lower().startswith('primary'):
                # add more conditions here for different cases 
                #(e.g. when simply a key is defined, or no key is defined)
                read_mode=0
                sio.seek(sio.tell()-1) # delete last comma
                sio.write('\n')
                continue
            colheader = re.findall(reg1,line)
            for col in colheader:
                sio.write(col.strip())
                sio.write(',')

        # Filling up the data -same as @firelynx's code
        elif read_mode ==2:
            for match in reg2.finditer(line): 
                newline = match.group("val")
                #print(newline)
                newline = newline.strip(' ()')
                #print(newline)
                newline = newline.replace('`', '')
                #print(newline)
                sio.write(newline)
                sio.write("\n")
            #if line.endswith(';'):
                #print("END OF INSERT")
sio.seek(0)
with open (PATH_TO_DATA+'ukwiki-20180920-langlinks.csv', 'w', encoding='ISO-8859-1') as fd:
    shutil.copyfileobj(sio, fd,-1)

In [ ]:
df_uk_langlinks = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks.csv', encoding='ISO-8859-1', quotechar="'", escapechar ="\\")
df_uk_langlinks

In [ ]:
df_uk_en_langlinks = df_uk_langlinks[df_uk_langlinks.ll_lang == 'en']
df_uk_en_langlinks.shape

In [ ]:
df_uk_en_langlinks.to_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', 
                     index = False, encoding='UTF-16', quotechar="\"", sep ="^")                          quotechar="'", escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
df_en_langlinks = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-langlinks_en.csv', encoding='UTF-16', sep="^")

In [ ]:
df_en_langlinks = df_en_langlinks[pd.notnull(df_en_langlinks['ll_title'])]
df_en_langlinks.head()

In [ ]:
df_en_langlinks.to_csv(PATH_TO_DATA + data_time + "-langlinks_uk_en.csv", index = False, 
                      encoding='ISO-8859-1', quotechar="'", 
                      escapechar ="\\", quoting=csv.QUOTE_NONNUMERIC)

In [6]:
df_uk_full = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-pages-links.csv')

In [42]:
df_articles.head()

,id,link_id,link_val,is_red_link
3,13,884280,Об'єкт (філософія),False
4,13,21623,Ератосфен,False
5,13,274944,географ,False
6,13,26194,Клавдій Птолемей,False
7,13,20459,Відродження,False


In [45]:
df_articles[df_articles['link_val']=='Католицизм']

,id,link_id,link_val,is_red_link
46505,1083,10732,Католицизм,False
69128,1348,10732,Католицизм,False
110523,1600,10732,Католицизм,False
116834,1624,10732,Католицизм,False
126687,1692,10732,Католицизм,False
128672,1707,10732,Католицизм,False
137094,1765,10732,Католицизм,False
137236,1765,10732,Католицизм,False
137303,1765,10732,Католицизм,False
142457,1784,10732,Католицизм,False


### Files needed for final processing:
### 1. 'ukwiki-20180920-pages-links.csv'
### 2. "ukwiki-" + data_time + "-id_name.csv"
### 3. '20180920-langlinks_uk_en.csv'
### 4. 'ukwiki-20180920-all_reslinks.csv'

In [69]:
df_uk_full = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-pages-links.csv')
print(df_uk_full.shape)
df_uk_full.head()

(22693778, 4)


,id,link_id,link_val,is_red_link
0,13,884280,Об'єкт (філософія),False
1,13,21623,Ератосфен,False
2,13,274944,географ,False
3,13,26194,Клавдій Птолемей,False
4,13,20459,Відродження,False


In [70]:
df_uk_blue = df_uk_full[df_uk_full['is_red_link']==False][['id','link_id']]

print('blue links:')
print(df_uk_blue.shape)
print(df_uk_blue.columns)
print()

blue links:
(19818824, 2)
Index(['id', 'link_id'], dtype='object')



In [71]:
df_uk_red = pd.read_csv(PATH_TO_DATA+'ukwiki-20180920-all_reslinks.csv', encoding='UTF-8', quotechar="\"")

In [72]:
df_uk_red.head()

,id,link_val
0,13,Географія розвитку
1,13,Географія часу
2,13,Туристична географія
3,584,рентгенівська спектроскопія поглинання
4,584,рентгенівська фото електронна спектроскопія


In [73]:
df_uk_red[df_uk_red['link_val']=='Москва ']

,id,link_val


In [74]:
# article names
df_uk_name = pd.read_csv(PATH_TO_DATA+"ukwiki-" + data_time + "-id_name.csv", encoding='utf-8', sep="^")

print(df_uk_name.shape)
print(df_uk_name.columns)
df_uk_name.head()

(819212, 3)
Index(['id', 'title', 'length'], dtype='object')


,id,title,length
0,3,Головна сторінка,3197
1,13,Географія,16843
2,584,Атом,35112
3,585,Мільярд,1759
4,586,Ядро,8064


In [75]:
# find maximum id among blue links
max_id = np.max(df_uk_name['id'])

# find unique titles among red links, create df with

red_links = df_uk_red['link_val'].unique()

red_links_ids = pd.DataFrame({'title': red_links, 
                               'id': np.arange(max_id+1, len(red_links)+max_id+1)})

red_links_ids.head()

,id,title
0,2817993,Географія розвитку
1,2817994,Географія часу
2,2817995,Туристична географія
3,2817996,рентгенівська спектроскопія поглинання
4,2817997,рентгенівська фото електронна спектроскопія


In [76]:
df_uk_red_ext = df_uk_red.merge(right = red_links_ids, left_on = 'link_val', right_on = 'title', how = 'right')
df_uk_red_ext = df_uk_red_ext[['id_x', 'link_val', 'id_y']]
df_uk_red_ext.columns = ['id_parent', 'red_link_name', 'red_link_id']
print(df_uk_red_ext.shape)
df_uk_red_ext.head()

(2872272, 3)


,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996


In [77]:
df_uk_red_ext.to_csv(PATH_TO_DATA+'all_uk_redlinks_with_parent_ids.csv')

In [15]:
# Load uk-en correpondences
fn = PATH_TO_DATA+'20180920-langlinks_uk_en.csv'
df_link = pd.read_csv(fn,  encoding='UTF-8', quotechar='\'')
print(df_link.shape)
print(df_link.columns)
df_link.head()

(599612, 3)
Index(['ll_from', 'll_lang', 'll_title'], dtype='object')


,ll_from,ll_lang,ll_title
0,2112530,en,! (The Dismemberment Plan album)
1,1366003,en,! (disambiguation)
2,2141483,en,!!
3,425480,en,!!!
4,848226,en,!Action Pact!


In [14]:
df_uk_red_ext = pd.read_csv(PATH_TO_DATA+'all_uk_redlinks_with_parent_ids.csv')
print(df_uk_red_ext.shape)
df_uk_red_ext.head()

(2872272, 4)


,Unnamed: 0,id_parent,red_link_name,red_link_id
0,0,13,Географія розвитку,2817993
1,1,13,Географія часу,2817994
2,2,1928599,Географія часу,2817994
3,3,13,Туристична географія,2817995
4,4,584,рентгенівська спектроскопія поглинання,2817996


In [16]:
# add en correspondences to uk parent pages for red links
df_uk_red_corresp = df_uk_red_ext.merge(right = df_link, left_on = 'id_parent', right_on = 'll_from', how = 'left')
df_uk_red_corresp.head()

,Unnamed: 0,id_parent,red_link_name,red_link_id,ll_from,ll_lang,ll_title
0,0,13,Географія розвитку,2817993,13.0,en,Geography
1,1,13,Географія часу,2817994,13.0,en,Geography
2,2,1928599,Географія часу,2817994,1928599.0,en,Geography (disambiguation)
3,3,13,Туристична географія,2817995,13.0,en,Geography
4,4,584,рентгенівська спектроскопія поглинання,2817996,584.0,en,Atom


In [17]:
df_uk_red_corresp = df_uk_red_corresp[pd.notnull(df_uk_red_corresp['ll_from'])]
df_uk_red_corresp.head()

,Unnamed: 0,id_parent,red_link_name,red_link_id,ll_from,ll_lang,ll_title
0,0,13,Географія розвитку,2817993,13.0,en,Geography
1,1,13,Географія часу,2817994,13.0,en,Geography
2,2,1928599,Географія часу,2817994,1928599.0,en,Geography (disambiguation)
3,3,13,Туристична географія,2817995,13.0,en,Geography
4,4,584,рентгенівська спектроскопія поглинання,2817996,584.0,en,Atom


In [18]:
df_uk_red_corresp = df_uk_red_corresp[['id_parent','red_link_name', 'red_link_id', 'll_title']]

In [19]:
df_uk_red_corresp.to_csv(PATH_TO_DATA+'df_allukredlinks_withentitle.csv', header=True, index=False)

In [82]:
df_uk_red_corresp = df_uk_red_corresp[['id_parent','red_link_name', 'red_link_id']]
df_uk_red_corresp = df_uk_red_corresp.sort_values(by = 'red_link_id')
df_uk_red_corresp = df_uk_red_corresp.drop_duplicates(keep = 'first')
df_uk_red_corresp = df_uk_red_corresp.dropna()
df_uk_red_corresp

,id_parent,red_link_name,red_link_id
0,13,Географія розвитку,2817993
1,13,Географія часу,2817994
2,1928599,Географія часу,2817994
3,13,Туристична географія,2817995
4,584,рентгенівська спектроскопія поглинання,2817996
5,584,рентгенівська фото електронна спектроскопія,2817997
11,2328123,мезолімбічний шлях,2817999
10,1209913,мезолімбічний шлях,2817999
8,42037,мезолімбічний шлях,2817999
9,427773,мезолімбічний шлях,2817999


In [83]:
# for every red link calculate number of incoming links
df_red_links_most_common = df_uk_red_corresp['red_link_id'].value_counts().reset_index()
df_red_links_most_common.head()

,index,red_link_id
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639


In [84]:
df_red_links_most_common.columns = ['red_link_id_', 'count']
df_red_links_most_common.head()

,red_link_id_,count
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639


In [85]:
df_red_links_most_common = df_red_links_most_common[(df_red_links_most_common['count'] >= 25)]
df_red_links_most_common

,red_link_id_,count
0,2863300,941
1,2969411,848
2,4169378,760
3,3060807,687
4,3512597,639
5,2826629,502
6,2826628,502
7,2826630,502
8,2826631,495
9,2825668,474


In [86]:
result = df_uk_red_corresp.merge(right = df_red_links_most_common,
                                 left_on = 'red_link_id', right_on = 'red_link_id_', how = 'inner')

In [87]:
result

,id_parent,red_link_name,red_link_id,red_link_id_,count
0,2579654,Каунт Бейсі,2818114,2818114,43
1,2543496,Каунт Бейсі,2818114,2818114,43
2,2543162,Каунт Бейсі,2818114,2818114,43
3,2538289,Каунт Бейсі,2818114,2818114,43
4,2581857,Каунт Бейсі,2818114,2818114,43
5,2530258,Каунт Бейсі,2818114,2818114,43
6,2525341,Каунт Бейсі,2818114,2818114,43
7,2389752,Каунт Бейсі,2818114,2818114,43
8,2535297,Каунт Бейсі,2818114,2818114,43
9,2590135,Каунт Бейсі,2818114,2818114,43


In [88]:
result_red_links_uk = result['red_link_name'].unique()
df_result_red_links_uk = pd.DataFrame(result_red_links_uk, columns=['red_link_name'])

# save red links to file
df_result_red_links_uk.to_csv(PATH_TO_DATA+'df_sample_red_links_uk.csv', header=True, index=False)